In [1]:
from crud import models
from crud import schemas
from crud.database import SessionLocal, engine
from sqlalchemy import MetaData, Table
import csv
import os
from collections import OrderedDict
import json
import numpy as np
import pandas as pd

In [2]:
db = SessionLocal()
CSV_DIR = './csv/'
JSON_DIR = './scraping_citaty_info/'
all_json_files = os.listdir(JSON_DIR)

pd.set_option('display.max_rows', 5000)

In [3]:
print(all_json_files)

['1-4448.json', '10000-100000.json', '100001-200000.json', '200000-300000.json', '300000-347314.json', '347315-400000.json', '400001-499999.json', '4449-10000.json', '500000-573191.json']


In [4]:
data = []

for current_json_file in all_json_files:
    with open(JSON_DIR + current_json_file, encoding="utf8") as json_file:
        data += json.load(json_file)
        # в data находится List<Dict>

при конверсии:
- удаляем tag5 и выше - слишком мало записей, такое тегирование излишне
- удалем character10 и выше - слишком мало записей

In [5]:
for dict_counter, dict_value in enumerate(data):
    for counter, value in enumerate(data[dict_counter]['characters']):
        if counter >= 10:
            continue
        data[dict_counter]['character' + str(counter)] = value
    for counter, value in enumerate(data[dict_counter]['tags']):
        if counter >= 5:
            continue
        data[dict_counter]['tag' + str(counter)] = value
    if data[dict_counter]['author2'] is not None:
        if ('цитаты' in data[dict_counter]['author2'].lower()):
            data[dict_counter]['tag5'] = data[dict_counter]['author2']
            data[dict_counter]['author2'] = ""
        else:
            data[dict_counter]['author'] = data[dict_counter]['author2']

In [6]:
df = pd.DataFrame(data)

In [7]:
del(data)

In [8]:
df.shape

(449664, 31)

In [9]:
df.columns

Index(['url', 'text', 'author', 'author2', 'book', 'characters', 'tags',
       'movie', 'anime', 'series', 'season', 'episode', 'band', 'song', 'game',
       'character0', 'tag0', 'tag1', 'character1', 'tag5', 'tag2', 'tag3',
       'tag4', 'character2', 'character3', 'character4', 'character5',
       'character6', 'character7', 'character8', 'character9'],
      dtype='object')

In [10]:
df.drop(labels=['url', 'author2', 'characters', 'tags'], axis=1, inplace=True)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 449664 entries, 0 to 449663
Data columns (total 27 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   text        449664 non-null  object
 1   author      274849 non-null  object
 2   book        158945 non-null  object
 3   movie       40368 non-null   object
 4   anime       10169 non-null   object
 5   series      45375 non-null   object
 6   season      42495 non-null   object
 7   episode     40741 non-null   object
 8   band        25636 non-null   object
 9   song        25635 non-null   object
 10  game        10308 non-null   object
 11  character0  165906 non-null  object
 12  tag0        368658 non-null  object
 13  tag1        232585 non-null  object
 14  character1  42059 non-null   object
 15  tag5        108965 non-null  object
 16  tag2        113769 non-null  object
 17  tag3        44991 non-null   object
 18  tag4        12694 non-null   object
 19  character2  3086 non-nu

In [12]:
# проверка на дупликаты
len(df['text'])-len(df['text'].drop_duplicates())

349

In [13]:
df.drop_duplicates(subset=['text'], keep='first', inplace=True)

In [14]:
# проверка на дупликаты
len(df['text'])-len(df['text'].drop_duplicates())

0

In [15]:
df.reset_index(inplace=True, drop=True)

In [16]:
df_spaces = df.copy()

In [17]:
df_spaces.fillna("", inplace=True)

#### Создание таблиц

- quotes - **done**
- проставить category_id - **done**
- authors - **done**
- проставить  и сразу связь с quotes **done**
- проставить категории в quotes **done**
- выгрузить quotes и authors **done**
- tags **done**
- выгрузить tags **done**
- сопряжение quote_tag **done**
- sources **done**
- сопряжение quotes_sources **done**

создание quotes

In [18]:
def make_category(book, movie, anime, series, band, song, game):
    if book != "":
        return 3
    elif movie != "":
        return 5
    elif anime != "":
        return 6
    elif series != "":
        return 7
    elif band != "":
        return 11
    elif song != "":
        return 11
    elif game != "":
        return 12 
    else:
        return 1

In [19]:
df_spaces['category_id'] = 1

In [20]:
df_spaces['category_id'] = df_spaces.apply(lambda row: make_category(row['book'], row['movie'], row['anime'], 
                                                                     row['series'], row['band'], row['song'], 
                                                                     row['game']), axis=1)

In [21]:
df_spaces['category_id'] = df_spaces['category_id'].astype(int)

In [22]:
df_spaces.category_id.value_counts()

3     158846
1     158698
7      45363
5      40317
11     25627
12     10303
6      10161
Name: category_id, dtype: int64

In [23]:
quotes = df_spaces[['text', 'category_id']].copy()
quotes.columns = ['content', 'category_id']
quotes['id'] = 1
quotes['language_id'] = 1
quotes['author_id'] = 0
quotes['rating_plus'] = 0
quotes['rating_minus'] = 0

In [24]:
# последоваиельные индексы для таблиц

range_ids = np.arange(1, quotes.shape[0]+1)
quotes['id'] = range_ids

In [25]:
# проверка на дупликаты
len(quotes['content'])-len(quotes['content'].drop_duplicates())

0

создание authors

In [26]:
authors_list = df_spaces['author'].copy()
authors_list.drop_duplicates(keep='first', inplace=True)

In [27]:
authors = pd.DataFrame(columns = ['id', 'language_id', 'emoji', 'avatar', 'name', 'about', 'description_id'])
authors['name'] = authors_list
range_ids = np.arange(1, authors.shape[0]+1)
authors['id'] = range_ids
authors.language_id = 1
authors.emoji = ""
authors.avatar = ""
authors.description_id = 0

authors['about'] = authors.apply(lambda row: 'Цитаты ' + row['name'] + 'а' , axis=1)
authors.reset_index(inplace=True, drop=True)

In [28]:
# проверка на дупликаты
len(authors['name'])-len(authors['name'].drop_duplicates())

0

Создание индексов для authors

In [29]:
# создание словаря авторов

authors_dict = {}

for row in range(len(authors)): 
    authors_dict[authors.loc[row, 'name']] = authors.loc[row, 'id']

In [30]:
quotes['author_id'] = df_spaces.apply(lambda row: authors_dict[row['author']] , axis=1)

Сохранение authors и quotes

In [31]:
authors.to_csv(path_or_buf="author.csv", index=False)

In [32]:
quotes.to_csv(path_or_buf="quote.csv", index=False)

создание tags

In [33]:
all_tags = []
for i in range(6):
    all_tags += list(df_spaces['tag'+str(i)])
    
all_tags_series = pd.Series(all_tags)
all_tags_series.drop_duplicates(keep='first', inplace=True)
all_tags_series.reset_index(inplace=True, drop=True)

In [34]:
# убираем элемент с пустым значением

index_zero = all_tags_series[all_tags_series == ""].index[0]
all_tags_series.drop(all_tags_series.index[[index_zero]], inplace=True)
all_tags_series.reset_index(inplace=True, drop=True)

In [35]:
tags = pd.DataFrame(columns = ['id', 'language_id', 'emoji', 'avatar', 'name', 'about', 'description_id'])
tags['name'] = all_tags_series
range_ids = np.arange(1, tags.shape[0]+1)
tags['id'] = range_ids
tags.language_id = 1
tags.emoji = ""
tags.avatar = ""
tags.description_id = 0

tags['about'] = tags.apply(lambda row: 'Цитаты про ' + row['name'], axis=1)
tags.reset_index(inplace=True, drop=True)

In [36]:
def rework_tags_name(name, about):
    if 'цитаты' in name:
        about = name
#         name = name.replace('цитаты', '')
#         name = name.replace(' ', '')
    elif name=='':
        about = ''
    return pd.Series([name, about], index=['name', 'about'])

In [37]:
tags[['name', 'about']] = tags.apply(lambda row: rework_tags_name(row['name'], row['about']), axis=1)

In [38]:
tags.reset_index(inplace=True, drop=True)

In [39]:
# проверка на дупликаты
len(tags['name'])-len(tags['name'].drop_duplicates())

0

In [40]:
tags.to_csv(path_or_buf="tag.csv", index=False)

Таблица sources

In [41]:
# сделаем индекс цитат

df_spaces['id'] = quotes['id']

In [42]:
list_possible_sources = ['book', 'movie', 'anime', 'series', 'season', 'episode', 'band', 'song', 'game']
characters = [f'character{i}' for i in range(10)]
list_possible_sources += characters
list_possible_sources_ids = [3, 5, 6, 7, 8, 9, 10, 11, 12, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]

In [43]:
sources_tables_list = []

for counter, source in enumerate(list_possible_sources):
    current_table = pd.DataFrame(columns = ['id', 'language_id', 'category_id', 'emoji', 
                                            'avatar', 'name', 'about', 'description_id'])
    current_series = df_spaces[source]
    current_series.drop_duplicates(keep='first', inplace=True)
    current_table['name'] = current_series
    current_table['language_id'] = 1
    current_table['emoji'] = ""
    current_table['avatar'] = ""
    current_table['description_id'] = 0
    current_table['category_id'] = list_possible_sources_ids[counter]
    
    print(counter, source, list_possible_sources_ids[counter])
    
    current_table['about'] = current_table.apply(lambda row: 'Цитаты ' + row['name'], axis=1)
    
    sources_tables_list.append(current_table)

0 book 3
1 movie 5
2 anime 6
3 series 7
4 season 8
5 episode 9
6 band 10
7 song 11
8 game 12
9 character0 4
10 character1 4
11 character2 4
12 character3 4
13 character4 4
14 character5 4
15 character6 4
16 character7 4
17 character8 4
18 character9 4


In [44]:
sources = pd.concat(sources_tables_list)

range_ids = np.arange(1, sources.shape[0]+1)
sources['id'] = range_ids

sources.reset_index(inplace=True, drop=True)

In [45]:
# проверка на дупликаты
len(sources['name'])-len(sources['name'].drop_duplicates())

14459

In [46]:
sources.drop_duplicates(subset=['name'], keep='first', inplace=True)

In [47]:
# проверка на дупликаты
len(sources['name'])-len(sources['name'].drop_duplicates())

0

In [48]:
# удаление пустого сорца

sources.drop(sources[sources.name == ""].index, inplace=True)

In [49]:
sources.reset_index(inplace=True, drop=True)
sources.to_csv(path_or_buf="source.csv", index=False)

### До этого места можно запускать всё на исполнение быстро

Сопряжение quote_tag

In [106]:
# создание словаря тегов

tags_dict = {}

for row in range(len(tags)): 
    tags_dict[tags.loc[row, 'name']] = tags.loc[row, 'id']

In [113]:
# на 450к цитат и 4к тегов этот блок работет примерно 40 минут

quote_tag = pd.DataFrame(columns = ['id', 'quote_id', 'tag_id'])

for i in range(422360, len(df_spaces)):
    for j in range(6):
        if df_spaces.loc[i, 'tag'+str(j)] != "":
            quote_tag = quote_tag.append({'quote_id': df_spaces.loc[i, 'id'], 
                                          'tag_id': tags_dict[df_spaces.loc[i, 'tag'+str(j)]]}, ignore_index=True)

In [114]:
quote_tag['quote_id'] = quote_tag['quote_id'].astype(int)
quote_tag['tag_id'] = quote_tag['tag_id'].astype(int)
range_ids = np.arange(1, quote_tag.shape[0]+1)
quote_tag['id'] = range_ids
quote_tag.reset_index(inplace=True, drop=True)

quote_tag.to_csv(path_or_buf="quote_tag1.csv", index=False)  

In [115]:
# перестройка индексов с двух загрузок

# quote_tag = pd.read_csv("quote_tag0.csv")
# quote_tag['tag_id'] = quote_tag['tag_id'].astype(int)
# range_ids = np.arange(1, quote_tag.shape[0]+1)
# quote_tag['id'] = range_ids
# quote_tag.to_csv(path_or_buf="quote_tag.csv", index=False)  

Сопряжение quotes_sources

In [50]:
sources_dict = {}

for row in range(len(sources)): 
    sources_dict[sources.loc[row, 'name']] = sources.loc[row, 'id']

In [51]:
quote_source = pd.DataFrame(columns = ['id', 'quote_id', 'source_id'])

for i in range(len(df_spaces)):
    for counter, source in enumerate(list_possible_sources):
        if df_spaces.loc[i, source] != "":
            quote_source = quote_source.append({'quote_id': df_spaces.loc[i, 'id'], 
                                          'source_id': sources_dict[df_spaces.loc[i, source]]}, ignore_index=True)

In [52]:
quote_source

,id,quote_id,source_id
0,NaN,1.0,23407.0
1,NaN,1.0,42791.0
2,NaN,2.0,23407.0
3,NaN,2.0,42791.0
4,NaN,3.0,2.0
...,...,...,...
611106,NaN,449284.0,11030.0
611107,NaN,449284.0,42821.0
611108,NaN,449307.0,503.0
611109,NaN,449308.0,503.0


In [54]:
quote_source['quote_id'] = quote_source['quote_id'].astype(int)
quote_source['source_id'] = quote_source['source_id'].astype(int)
range_ids = np.arange(1, quote_source.shape[0]+1)
quote_source['id'] = range_ids
quote_source.reset_index(inplace=True, drop=True)

quote_source.to_csv(path_or_buf="quote_source.csv", index=False)  

### Полное исполнение до сюда

In [ ]:
df.loc[(df.author != '') & (df.author2 != '')]

In [ ]:
df.loc[60744:60746]

In [211]:
df.author.value_counts().to_frame().head(5000)

,author
Петр Квятковский,3183
Станислав Ежи Лец,2935
Евгений Витальевич Антонюк,2230
Эрих Мария Ремарк,2052
Стивен Кинг,1867
Макс Фрай,1705
Айшек Норам,1518
Олег Рой,1456
Уильям Шекспир,1454
Эльчин Сафарли,1393
